# Oráculo de Brito: Análise de Conflito Urbano (V5)
## Pipeline como Código
Este notebook orquestra a pipeline de análise. A lógica de construção da base de conhecimento foi abstraída para o script `src/build_knowledge_base.py` para robustez e manutenibilidade.

### Passo 1: Construir a Base de Conhecimento
A célula abaixo executa o script `build_knowledge_base.py`, que realiza três etapas: 
1. Gera o `schema_v5.ttl`
2. Gera o `kb_v5_final.ttl` (com instâncias)
3. Executa a inferência e gera o `kb_v5_inferido.ttl`

### Passo 2: Carregar o Grafo Inferido e Preparar Consultas
Com os dados gerados, carregamos o grafo final (que já contém as triplas inferidas) e preparamos nosso motor de consultas.

In [2]:
from rdflib import Graph
import sys
sys.path.insert(0, '..')
from src.sparql_queries import SPARQLQueryEngine

g_inferido = Graph()
g_inferido.parse("../data/kb_conflito_v5_inferido.ttl", format="turtle")

engine = SPARQLQueryEngine(g_inferido)

print(f"✓ Grafo inferido carregado com {len(g_inferido)} triplas. Pronto para análise.")

✓ Grafo inferido carregado com 309 triplas. Pronto para análise.


### Passo 3: Análise - Extrair Narrativas de Conflito
Executamos as consultas SPARQL para provar as hipóteses de conflito.

In [3]:
print("--- ANÁLISE DE CONFLITO V5 ---")

# Consulta 1: Conflito Normativo Explícito
print("[CONSULTA 1: Conflito Normativo Explícito]")
conflitos = engine.query_normative_conflict()
if conflitos:
    for c in conflitos:
        print(f"  ✓ Conflito PROVADO entre '{c['norma1_label']}' e '{c['norma2_label']}'")
else:
    print("  - Nenhum conflito normativo explícito encontrado.")

--- ANÁLISE DE CONFLITO V5 ---
[CONSULTA 1: Conflito Normativo Explícito]
  ✓ Conflito PROVADO entre 'Lei do PREZEIS (1995)' e 'Lei do Remembramento (2020)'


### Passo 4: Visualização do Grafo
Geramos uma visualização interativa do grafo inferido para exploração visual.

In [4]:
from pyvis.network import Network
from IPython.display import HTML
from rdflib import RDFS

net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed=True)

# Adiciona apenas as triplas do nosso namespace para clareza
for s, p, o in g_inferido:
    if 'recife.leg.br' in str(s) and 'recife.leg.br' in str(o):
        s_label = g_inferido.value(s, RDFS.label) or s.split('#')[-1]
        o_label = g_inferido.value(o, RDFS.label) or o.split('#')[-1]
        net.add_node(str(s), label=str(s_label))
        net.add_node(str(o), label=str(o_label))
        
        edge_label = p.split('#')[-1]
        color = "#FF6347" if edge_label == "conflitaCom" else "#FFFFFF"
        width = 3 if edge_label == "conflitaCom" else 1
        net.add_edge(str(s), str(o), label=edge_label, color=color, width=width)

output_filename = "oraculo_de_brito_visualizacao_v5.html"
net.write_html(output_filename, notebook=True)
print(f"✓ Visualização V5 salva como '{output_filename}'.")
HTML(filename=output_filename)

✓ Visualização V5 salva como 'oraculo_de_brito_visualizacao_v5.html'.
